# Numerical Integration and Differentiation

This notebook illustrates how to perform numerical integration and differentiation. 

There are several packages for doing this. Here, the focus is on QuadGK (https://github.com/JuliaMath/QuadGK.jl) and ForwardDiff (https://github.com/JuliaDiff/ForwardDiff.jl).

While QuadGK relies on well established ("standard") ways of numerical integration, ForwardDiff uses a more novel approach. (Calculus.jl provides similar functionality, applying a more traditional methods.)

## Load Packages

In [1]:
using QuadGK, ForwardDiff

include("printmat.jl")

printmatDate

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Numerical Integration

## The Pdf of the N(μ,σ²) Distribution 

In [3]:
function ϕNS(x,μ=0,σ²=1)       #pdf of N(μ,σ²), NS for non-standard
    σ = sqrt(σ²)
    z = (x - μ)/σ
    pdf = exp(-0.5*z^2)/(sqrt(2*pi)*σ)    
    return pdf
end

ϕNS (generic function with 3 methods)

In [4]:
x  = -3:0.1:3
xb = x[x.<=1.645]

plot1 = plot(x,ϕNS.(x),color=:red,linewidth=2,legend=nothing)
plot!(xb,ϕNS.(xb),color=:red,linewidth=2,legend=nothing,fill=(0,:red))
title!("pdf of N(0,1)")
xlabel!("x")
ylabel!("")
annotate!(1.75,0.25,"the area covers\n up to x=1.645")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 pdf of N(0,1) 
 
 
 x 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 32.2221,356.415 41.357,355.163 50.4919,353.546 59.6267,351.483 68.7616,348.878 77.8965,345.629 87.0314,341.621 96.1663,336.735 105.301,330.848 114.436,323.84 
 123.571,315.598 132.706,306.024 141.841,295.042 150.976,282.606 160.11,268.711 169.245,253.395 178.38,236.75 187.515,218.926 196.65,200.134 205.785,180.642 
 214.92,160.778 224.055,140.917 233.189,121.475 242.324,102.893 251.459,85.6212 260.594,70.104 269.729,56.7577 278.864,45.9539 287.999,38.0022 297.134,33.1348 
 306.269,31.4961 315.403,33.1348 324.538,38.0022 333.673,45.9539 342.808,56.7577 351.943,70.104 361.078,85.6212 370.213,102.893 379.348,121.475 388.482,140.917 
 397.617,160.778 406.752,180.642 415.887,200.134 425.022,218.926 434.157,236.75 443.292,253.395 452.427,268.711 461.562,282.606 470.696,295.042 479.831,306.024 
 488.966,315.598 498.101,323.84 507.236,330.848 516.371,336.735 525.506,341.621 534.641,345.629 543.775,348.878 552.91,351.483 562.045,353.546 571.18,355.163 
 580.315,356.415 
 "/>
<polygon clip-path="url(#clip02)" points="
32.2221,356.415 41.357,355.163 50.4919,353.546 59.6267,351.483 68.7616,348.878 77.8965,345.629 87.0314,341.621 96.1663,336.735 105.301,330.848 114.436,323.84 
 123.571,315.598 132.706,306.024 141.841,295.042 150.976,282.606 160.11,268.711 169.245,253.395 178.38,236.75 187.515,218.926 196.65,200.134 205.785,180.642 
 214.92,160.778 224.055,140.917 233.189,121.475 242.324,102.893 251.459,85.6212 260.594,70.104 269.729,56.7577 278.864,45.9539 287.999,38.0022 297.134,33.1348 
 306.269,31.4961 315.403,33.1348 324.538,38.0022 333.673,45.9539 342.808,56.7577 351.943,70.104 361.078,85.6212 370.213,102.893 379.348,121.475 388.482,140.917 
 397.617,160.778 406.752,180.642 415.887,200.134 425.022,218.926 434.157,236.75 443.292,253.395 452.427,268.711 452.427,360.065 443.292,360.065 434.157,360.065 
 425.022,360.065 415.887,360.065 406.752,360.065 397.617,360.065 388.482,360.065 379.348,360.065 370.213,360.065 361.078,360.065 351.943,360.065 342.808,360.065 
 333.673,360.065 324.538,360.065 315.403,360.065 306.269,360.065 297.134,360.065 287.999,360.065 278.864,360.065 269.729,360.065 260.594,360.065 251.459,360.065 
 242.324,360.065 233.189,360.065 224.055,360.065 214.92,360.065 205.785,360.065 196.65,360.065 187.515,360.065 178.38,360.065 169.245,360.065 160.11,360.065 
 150.976,360.065 141.841,360.065 132.706,360.065 123.571,360.065 114.436,360.065 105.301,360.065 96.1663,360.065 87.0314,360.065 77.8965,360.065 68.7616,360.065 
 59.6267,360.065 50.4919,360.065 41.357,360.065 32.2221,360.065 
 " fill="#ff0000" fill-opacity="1"/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 32.2221,356.415 41.357,355.163 50.4919,353.546 59.6267,351.483 68.7616,348.878 77.8965,345.629 87.0314,341.621 96.1663,336.735 105.301,330.848 114.436,323.84 
 123.571,315.598 132.706,306.024 141.841,295.042 150.976,282.606 160.11,268.711 169.245,253.395 178.38,236.75 187.515,218.926 196.65,200.134 205.785,180.642 
 214.92,160.778 224.055,140.917 233.189,121.475 242.324,102.893 251.459,85.6212 260.594,70.104 269.729,56.7577 278.864,45.9539 287.999,38.0022 297.134,33.1348 
 306.269,31.4961 315.403,33.1348 324.538,38.0022 333.673,45.9539 342.808,56.7577 351.943,70.104 361.078,85.6212 370.213,102.893 379.348,121.475 388.482,140.917 
 397.617,160.778 406.752,180.642 415.887,200.134 425.022,218.926 434.157,236.75 443.292,253.395 452.427,268.711 
 "/>
 
 the area covers 
 
 
 up to x=1.645

## Calculating Prob(x<=1.645)

In [5]:
cdf164, = QuadGK.quadgk(x->ϕNS(x),-Inf,1.645)
printlnPs("\nPr(x<=1.64) according to N(0,1): $(round(cdf164,3))")

printlnPs("\n...yes, there is a smarter way to do this calculations, but it's still a good illustration")


Pr(x<=1.64) according to N(0,1): 0.95

...yes, there is a smarter way to do this calculations, but it's still a good illustration


# Numerical Derivatives

Numerical derivatives can be calculated by a crude finite difference (see NumDer() below) or the much more sophisticated routines in the ForwardDiff.jl package. 

The function you want to differentiate must typically have only one argument (even if it is a vector). To turn a multi-argument function fn1(x,a) into a one-argument function transform it to an anonymous function by

```
x->fn1(x,a),
```
assuming that ```a``` has a value already. We can now differentiate wrt. ```x```.

In [6]:
function NumDer(fun,b0,h)           #crude function for a centered numerical derivative
    bminus = b0 .- h
    bplus  = b0 .+ h
    hh     = bplus - bminus
    fplus  = fun(bplus)
    fminus = fun(bminus)
    D      = (fplus-fminus)/hh
    return D
end

NumDer (generic function with 1 method)

In [7]:
function fn1(x,a)                 #a simple function, to be differentiated
   return (x - 1.1)^2 - a                
end

fn1 (generic function with 1 method)

In [8]:
x0 = 2

dydx_A = NumDer(x->fn1(x,0.5),x0,0.01)           #differentiate fn1(x,0.5) at x = x0
dydx_B = ForwardDiff.derivative(x->fn1(x,0.5),x0)

println("The derivative at x=$x0 is (from two different methods): ")
printmat([dydx_A dydx_B])

The derivative at x=2 is (from two different methods): 
     1.800     1.800



In [9]:
x = -3:6/99:6          #calculate the derivative at many points

dydx_A = [NumDer(x->fn1(x,0.5),x[i],0.01) for i=1:length(x)]        

dydx_B = [ForwardDiff.derivative(x->fn1(x,0.5),x[i]) for i=1:length(x)]

println("now lets plot this")

now lets plot this


In [10]:
plot(x,fn1.(x,0.5),color=:black,label="fn1()")
plot!(x,dydx_A,color=:red,line=(:dot,4),label="crude derivative")
plot!(x,dydx_B,color=:blue,line=(:dash,2),label="ForwardDiff pkg ")
title!("fn1() and its derivative")
xlabel!("x")
ylabel!("")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 fn1() and its derivative 
 
 
 x 
 
<polyline clip-path="url(#clip02)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 29.2591,103.707 32.9825,108.866 36.7058,113.949 40.4292,118.955 44.1525,123.884 47.8759,128.736 51.5992,133.512 55.3226,138.21 59.0459,142.832 62.7693,147.377 
 66.4926,151.845 70.216,156.236 73.9393,160.55 77.6627,164.788 81.386,168.949 85.1094,173.032 88.8327,177.039 92.5561,180.97 96.2794,184.823 100.003,188.599 
 103.726,192.299 107.449,195.922 111.173,199.468 114.896,202.937 118.62,206.329 122.343,209.645 126.066,212.884 129.79,216.045 133.513,219.13 137.236,222.139 
 140.96,225.07 144.683,227.924 148.406,230.702 152.13,233.403 155.853,236.027 159.576,238.574 163.3,241.044 167.023,243.438 170.746,245.754 174.47,247.994 
 178.193,250.157 181.916,252.243 185.64,254.252 189.363,256.185 193.087,258.04 196.81,259.819 200.533,261.521 204.257,263.146 207.98,264.694 211.703,266.166 
 215.427,267.56 219.15,268.878 222.873,270.119 226.597,271.283 230.32,272.37 234.043,273.381 237.767,274.314 241.49,275.171 245.213,275.951 248.937,276.654 
 252.66,277.28 256.384,277.83 260.107,278.302 263.83,278.698 267.554,279.017 271.277,279.259 275,279.424 278.724,279.512 282.447,279.524 286.17,279.458 
 289.894,279.316 293.617,279.097 297.34,278.801 301.064,278.429 304.787,277.979 308.51,277.453 312.234,276.85 315.957,276.17 319.68,275.413 323.404,274.579 
 327.127,273.669 330.851,272.681 334.574,271.617 338.297,270.476 342.021,269.258 345.744,267.964 349.467,266.592 353.191,265.144 356.914,263.619 360.637,262.017 
 364.361,260.338 368.084,258.582 371.807,256.749 375.531,254.84 379.254,252.854 382.977,250.791 386.701,248.651 390.424,246.434 394.147,244.141 397.871,241.77 
 401.594,239.323 405.318,236.799 409.041,234.198 412.764,231.52 416.488,228.766 420.211,225.934 423.934,223.026 427.658,220.041 431.381,216.979 435.104,213.84 
 438.828,210.625 442.551,207.332 446.274,203.963 449.998,200.517 453.721,196.994 457.444,193.394 461.168,189.717 464.891,185.964 468.614,182.134 472.338,178.227 
 476.061,174.243 479.785,170.182 483.508,166.044 487.231,161.83 490.955,157.538 494.678,153.17 498.401,148.725 502.125,144.203 505.848,139.605 509.571,134.929 
 513.295,130.177 517.018,125.348 520.741,120.442 524.465,115.459 528.188,110.399 531.911,105.263 535.635,100.049 539.358,94.7593 543.081,89.3922 546.805,83.9484 
 550.528,78.4277 554.252,72.8302 557.975,67.1558 561.698,61.4046 565.422,55.5766 569.145,49.6717 572.868,43.69 576.592,37.6314 580.315,31.4961 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:4; stroke-opacity:1; fill:none" stroke-dasharray="2, 4" points="
 29.2591,360.065 32.9825,358.798 36.7058,357.53 40.4292,356.262 44.1525,354.994 47.8759,353.726 51.5992,352.459 55.3226,351.191 59.0459,349.923 62.7693,348.655 
 66.4926,347.387 70.216,346.12 73.9393,344.852 77.6627,343.584 81.386,342.316 85.1094,341.048 88.8327,339.781 92.5561,338.513 96.2794,337.245 100.003,335.977 
 103.726,334.709 107.449,333.442 111.173,332.174 114.896,330.906 118.62,329.638 122.343,328.37 126.066,327.103 129.79,325.835 133.513,324.567 137.236,323.299 
 140.96,322.031 144.683,320.764 148.406,319.496 152.13,318.228 155.853,316.96 159.576,315.692 163.3,314.425 167.023,313.157 170.746,311.889 174.47,310.621 
 178.193,309.353 181.916,308.086 185.64,306.818 189.363,305.55 193.087,304.282 196.81,303.014 200.533,301.747 204.257,300.479 207.98,299.211 211.703,297.943 
 215.427,296.675 219.15,295.408 222.873,294.14 226.597,292.872 230.32,291.604 234.043,290.336 237.767,289.069 241.49,287.801 245.213,286.533 248.937,285.265 
 252.66,283.997 256.384,282.73 260.107,281.462 263.83,280.194 267.554,278.926 271.277,277.658 275,276.391 278.724,275.123 282.447,273.855 286.17,272.587 
 289.894,271.319 293.617,270.0

## (extra) Comments on Using the ForwardDiff Package

The ForwardDiff package applies an interesting approach to calculate derivatives, using a special number type ("dual numbers"). This means that your code must be able to handle such numbers. In most cases, that is not a problem, but you may have to watch out if you create arrays to store (intermediate?) results inside the function. See the examples below

In [11]:
function fnDoesNotWork(b,a)
    z = zeros(length(b))              #will not work with ForwardDiff, since
    for i = 1:length(z)               #z cannot store dual numbers
        z[i] = b[i]*i
    end    
    return sum(z) + a
end    

function fnDoesWork(b,a)
    z = zeros(eltype(b),length(b))   #will work with ForwardDiff, since
    for i = 1:length(z)              #when b is a dual number, so is z 
        z[i] = b[i]*i
    end    
    return sum(z) + a
end

fnDoesWork (generic function with 1 method)

In [12]:
b0 = [1.5;2]

try
    ForwardDiff.gradient(b->fnDoesNotWork(b,1),b0)
    println("using fnDoesNotWork(): ",dydx)
catch    
    dydx = ForwardDiff.gradient(b->fnDoesWork(b,1),b0)
    println("using fnDoesWork(): ",dydx)    
end

using fnDoesWork(): [1.0, 2.0]
